In [14]:
import importlib
from pathlib import Path

import numpy as np
import pandas as pd

import config
import tests
from modules import module

try:
    importlib.reload(module)  # reload module
except NameError:
    pass

try:
    importlib.reload(tests)  # reload module
except NameError:
    pass

try:
    importlib.reload(config)  # reload module
except NameError:
    pass

# Load DF

In [15]:
path = Path(config.path_raw_file)
try:
    df = pd.read_csv(path)
except UnicodeDecodeError:
    df = pd.read_excel(path)

# Clean DF

In [16]:
df_clean = (
    df
    # drop na
    .dropna(subset="From")
    .dropna(subset="To")
    # rename column
    .rename(columns=lambda c: module.clean_col_name(c))
    .rename(columns=module.to_rename)
    .assign(
        # clean city
        city=lambda df_: (df_["city"].str.title().str.strip()),
        # clean dob, if dob1 blank then dob2
        dob=lambda df_: np.where(
            df_["dob"].isna(), pd.to_datetime(df_["dob2"]), pd.to_datetime(df_["dob"])
        ),
        # clean start_date, end_date, fp_date
        start_date=lambda df_: pd.to_datetime(df_["start_date"]),
        end_date=lambda df_: pd.to_datetime(df_["end_date"]),
        fp_date=lambda df_: pd.to_datetime(df_["fp_date"]),
        # clean income
        income=lambda df_: df_["income"].astype(module.income_cat),
        # clean job
        job=lambda df_: module.clean_job(df_),
        # create age
        age=lambda df_: module.get_age(df_),
        # create id
        id=lambda df_: (
            df_["name"] + " " 
            + df_["dob"].dt.strftime("%Y-%m-%d") + " " 
            + df_["partner_street"].astype(str)
        ),
        # create membership code
        membership_code=lambda df_: module.get_membership_code(df_["product"]),
        # create membership duration
        membership_duration=lambda df_: module.get_membership_duration(df_["product"]),
    )
    # merge with membership mapping to obtain membership
    .merge(
        right=pd.read_excel(Path("input/membership_mapping.xlsx")),
        on="membership_code",
        how="left",
    )
    .assign(
        # make sure that all corporate is mapped
        is_cpt=lambda df_: module.assert_cpt_catched(df_),
        # create student center and area 
        center=lambda df_: module.clean_center(df_),
        area=lambda df_: module.clean_area(df_),
    )
    # create is next contract col
    # sort by id first
    .sort_values(["id", "end_date"], ascending=[True, False])
    .assign(is_next_contract=lambda df_: df_["id"].duplicated(keep="last"))
    # drop membership code na
    .loc[lambda df_: ~(df_["membership_code"].isna())]
    # filter staff
    .loc[lambda df_: df_["contract_type"].str.title() != "Employee"]
    .loc[lambda df_: df_["core_product"].str.title() != "Staff"]
    # drop date too old
    .loc[lambda df_: pd.to_datetime(df_["start_date"]).dt.year >= 2020]
    # drop membership code na
    .loc[lambda df_: ~(df_["membership_code"].isna())]
    # filter cancelled member and free member and non member
    .loc[
        lambda df_: ~(
            df_["membership_status"].str.title().isin(
                ["Cancelled Member", "Free Member", "Non Member"]
            )
        )
    ]
    # drop unused cols
    .drop(columns=module.to_drop)
    # sort column
    .sort_index(axis=1)
)

date_columns = [
    "jan_2023", "feb_2023", "mar_2023", "apr_2023", "may_2023", "jun_2023",
    "jul_2023", "aug_2023", "sep_2023", "oct_2023", "nov_2023", "dec_2023", 
    "jan_2024", "feb_2024",
]
for month in date_columns:
    activity_assignments = {
        f"active_{month}": lambda df_: module.is_active(
            df_, "start_date", "end_date", f"1 {month.replace('_', ' ')}"
    )}
    df_clean = df_clean.assign(**activity_assignments)

# Tests and Assertions

In [17]:
tests.test_all_centers_mapped(df)
tests.test_all_centers_are_filled(df_clean)
tests.test_all_areas_are_filled(df_clean)
tests.test_all_memberships_are_filled(df_clean)
tests.test_all_membership_mapped(df_clean)
tests.test_cpt_is_flagged(df_clean)
tests.test_cpt_in_cpt_center(df_clean)
tests.test_cpt_in_cpt_area(df_clean)
tests.test_noncpt_in_noncpt_center(df_clean)
tests.test_noncpt_in_noncpt_area(df_clean)

# Save DF

In [18]:
import os
file= df_clean
path= f'output/member_data_{Path(config.path_raw_file).stem}.xlsx'

if not os.path.exists(path):
    file.to_excel(path, index=False)
    print('File saved.')
else:
    print('File already exist.')

File saved.


# Experiment

In [19]:
# df.loc[df['Membership/Display Name'].str.lower().str.contains('ilt', na= False)].sort_values('From')

In [20]:
# df['Membership/Display Name'].unique()

In [21]:
# df_clean['is_cpt'].value_counts()

In [22]:
# df["Partner/Branch/Display Name"].unique()

In [23]:
# (df_clean
#     .loc[df_clean[""]]
# )

# mask = (
#     df_clean["addon_1"].astype(str)
#     + df_clean["addon_2"].astype(str)
#     + df_clean["addon_3"].astype(str)
# ).str.lower().str.contains("club")

# mask2 = df_clean["is_cpt"].isna()

# (df_clean
#     .loc[mask & mask2]
#     .loc[lambda df_: df_["fp_date"] >= pd.to_datetime("2023-01-01")]
#     .assign(fp_month=lambda df_: df_["fp_date"].dt.strftime("%Y-%m-01"))
#     .groupby(["fp_month", "is_renewal"])
#     .agg(count=("id", "nunique"))
#     .reset_index()
#     .pivot(index="fp_month", columns="is_renewal")
# )

In [24]:
# # (df_clean
# #     .loc[df_clean[""]]
# # )

# mask = (
#     df_clean["core_product"].astype(str)
# ).str.lower().str.contains("go")

# (df_clean
#     .loc[mask]
#     .loc[lambda df_: df_["fp_date"] >= pd.to_datetime("2023-01-01")]
#     .assign(fp_month=lambda df_: df_["fp_date"].dt.strftime("%Y-%m-01"))
#     .groupby(["fp_month", "is_renewal"])
#     .agg(count=("id", "nunique"))
#     .reset_index()
#     .pivot(index="fp_month", columns="is_renewal")
# )

In [25]:
# (
#     df_clean
#     # get jan 2024 fp
#     .loc[(df_clean["fp_date"].dt.month == 1) & (df_clean["fp_date"].dt.year == 2024)]
#     # get renewal
#     .loc[lambda df_: df_["is_next_contract"] == True]
#     # get relevant cols
#     .loc[
#         :,
#         [
#             "email",
#             "fp_date",
#             "id",
#             "is_renewal",
#             "membership_code",
#             "membership_duration",
#             "membership_status",
#             "partner_hobby",
#             "start_date",
#         ],
#     ]
#     # .to_excel("rep.xlsx")
#     # .loc[
#     #     lambda df_: df_["id"].str.contains(
#     #         "ADI NUGROHO|ELISA NOVRIYANTI MARPAUNG|ERLINA NOERPITASARI|NAUFAL PANDU IRSYADI"
#     #     )
#     # ]
# )